In [128]:
# Dependencies.
import pandas as pd
from config import client_id, client_secret
from params import key_signatures, modes, key_int
import os
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from pprint import pprint
import numpy as np

# Load variables to environment.
os.environ['SPOTIPY_CLIENT_ID'] = client_id
os.environ['SPOTIPY_CLIENT_SECRET'] = client_secret

In [129]:
# Function to obtain user's input.
def get_input():

    user_input = input('Enter the playlist URI: ')

    return user_input

In [130]:
# Function to perform API call and return results in JSON format.
def get_results(t, o):

    sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())

    # Check if string contains 'https', the ref['next'] will contain this if the list has more than 100 songs.
    if 'https' in t:

        # Split the string in URL using '/'
        split = t.split('/')

        # Split list index 5 will contain part of URI, multiple o by 100 for the offset in the function call.
        results = sp.playlist_tracks('spotify:playlist:' + split[5], offset = o * 100)

    else:
        results = sp.playlist_tracks(t)

    return results

In [131]:
# Function to create dict with data.
def create_df(results):

    # Empty dict to hold data from Spotify.
    data = {
        'track': [],
        'artist': [],
        'uri': []
    }

    # Loop control.
    loop = True

    # Results as json object.
    ref = results

    # Offset variable.
    offset = 0

    while loop:

        # Adds values to dict.
        for song in ref['items']:
            data['track'].append(song['track']['name'])
            data['artist'].append(song['track']['artists'][0]['name'])
            data['uri'].append(song['track']['uri'])

        # Check for ref['next'], if there's text then playlist has more items.
        if ref['next'] is not None:
            offset += 1
            ref = get_results(ref['next'], offset)
        
        # Terminate loop.
        else:
            loop = False

    return pd.DataFrame(data)

In [132]:
# Function to perform API call for audio features and return results in JSON format.
def get_features(t):

    sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials())

    results = sp.audio_features(t)

    return results

In [133]:
# Function to find the time key signatures of the song.
def features_finder(results):

    try:

        m = results[0]['mode']

        k = results[0]['key']

        t = results[0]['time_signature']

        b = results[0]['tempo']
    
    except IndexError:
        
        return np.NaN, np.NaN, np.NaN, np.NaN
    
    return m, k, t, b

In [134]:
results = get_results(get_input(), 1)
df = create_df(results)

uris = list(df['uri'])

md_col = []
ks_col = []
kint_col = []
ts_col = []
bpm_col = []

for uri in uris:
    response = get_features(uri)
    md, ks, ts, bpm = features_finder(response)

    md_col.append(md)
    ks_col.append(ks)
    kint_col.append(key_int[md][ks])
    ts_col.append(ts)
    bpm_col.append(bpm)


df['mode'] = md_col
df['key_signature'] = ks_col
df['key_integer'] = kint_col
df['time_signature'] = ts_col
df['bpm'] = bpm_col


df = df.dropna()
df

,track,artist,uri,mode,key_signature,key_integer,time_signature,bpm
0,Checkmate,Lamb of God,spotify:track:6q72OHGM81Zs5q5jx7YZLR,1,2,2,4,119.980
1,Inside Out,Five Finger Death Punch,spotify:track:4Htt3QaBWdLggq88rJI5MU,0,4,1,4,82.997
2,The End of All We Know,Bleed From Within,spotify:track:14EXZEog1hW97d6yz8ejih,0,6,3,3,149.406
3,Nero Forte,Slipknot,spotify:track:56fiFTRrSiHHH3gBeaTg2P,1,7,1,4,129.842
4,Noise,Nightwish,spotify:track:2a3sZJP4uTrNQbj0EdiUuV,1,2,2,4,76.976
...,...,...,...,...,...,...,...,...
56,Fade,Catch Your Breath,spotify:track:6GS7dWKyY8vCJHaThLZVEz,1,2,2,4,119.978
57,Counterfeit,Wolves At The Gate,spotify:track:0nER4VGgRAjyxtM9f1H43I,1,1,-5,4,95.009
58,The Red Baron,Sabaton,spotify:track:2nb7KVajlFlcJG2mAZiDuG,0,0,-3,4,95.047
59,MAYDAY (feat. Ryo from Crystal Lake),coldrain,spotify:track:2wJJNgD6GrKoQ435VxTRv6,0,10,-5,4,121.054


In [138]:
df.to_csv('output.csv', index = False)